### Embedding calculation using Model Registry deployment to SPCS

In [ ]:
!pip install sentence_transformers snowflake-ml-python==1.7.5

In [ ]:
!pip list

In [ ]:
from snowflake.ml.registry import registry
from sentence_transformers import SentenceTransformer
from snowflake.snowpark.context import get_active_session

In [ ]:
session = get_active_session()
reg = registry.Registry(session=session)

In [ ]:
model_name = "sentence_transformer_minilm"
image_repo_name = "embedding_images"
cp_name = "kipi_embedding_gpu_nv_s"
num_spcs_nodes = "4"
spcs_instance_family = "GPU_NV_S"
service_name = "kipi_embedding_service"

current_database = session.get_current_database().replace('"', '')
current_schema = session.get_current_schema().replace('"', '')
extended_image_repo_name = f"{current_database}.{current_schema}.{image_repo_name}"
extended_service_name = f'{current_database}.{current_schema}.{service_name}'

In [ ]:
# Take an example sentence transformer from HF
embed_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
# Have some sample input data
input_data = [
    "This is the first sentence.",
    "Here's another sentence for testing.",
    "The quick brown fox jumps over the lazy dog.",
    "I love coding and programming.",
    "Machine learning is an exciting field.",
    "Python is a popular programming language.",
    "I enjoy working with data.",
    "Deep learning models are powerful.",
    "Natural language processing is fascinating.",
    "I want to improve my NLP skills.",
]

In [ ]:
embeddings = embed_model.encode(input_data)
print(embeddings)

In [ ]:
_ = reg.log_model(
    embed_model,
    model_name=model_name,
    sample_input_data=input_data,
    pip_requirements=["sentence-transformers", "torch", "transformers"]
)

In [ ]:
m = reg.get_model(model_name)
version_df = m.show_versions()
version_df.head(100)

In [ ]:
last_version_name = version_df['name'].iloc[-1]
pip_model = m.version(last_version_name)
pip_model

In [ ]:
session.sql(f"alter compute pool if exists {cp_name} stop all").collect()
session.sql(f"drop compute pool if exists {cp_name}").collect()
session.sql(f"create compute pool {cp_name} min_nodes={num_spcs_nodes} max_nodes={num_spcs_nodes} instance_family={spcs_instance_family} auto_resume=True auto_suspend_secs=300").collect()
session.sql(f"describe compute pool {cp_name}").show()

In [ ]:
session.sql(f"create or replace image repository {extended_image_repo_name}").collect()

In [ ]:
pip_model.create_service(
    service_name=extended_service_name,
    service_compute_pool=cp_name,
    image_repo=extended_image_repo_name,
    ingress_enabled=True,
    gpu_requests="1",
    max_instances=int(num_spcs_nodes),
    build_external_access_integration="ALLOW_ALL_INTEGRATION"
)

In [ ]:
# See all services running a model
pip_model.list_services()

In [ ]:
session.sql(f"SELECT VALUE:status::VARCHAR as SERVICESTATUS, VALUE:message::VARCHAR as SERVICEMESSAGE FROM TABLE(FLATTEN(input => parse_json(system$get_service_status('{service_name}')), outer => true)) f").show(100)

In [ ]:
# Run on SPCS
pip_model.run(input_data, function_name="encode", service_name=service_name)

In [ ]:
session.sql(f"show endpoints in service {service_name}").collect()[0]["ingress_url"]

In [ ]:
pip_model.show_functions()

In [ ]:
feature_columns = [feature.name for feature in pip_model.show_functions()[0]["signature"].inputs]
feature_columns

In [ ]:
# Prepare data to be sent
data = {
    'data': []
}
for idx, x in enumerate(input_data):
    data['data'].append([idx, x])
print(data)